In [4]:
# Pandas pour manipuler les tableaux de données
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

# scikit learn pour les outils de machine learning
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

# librairies pour la visualisation de données
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# et quelques options visuelles
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set(style="whitegrid", color_codes=True)
sns.set(rc={'figure.figsize':(15,8)})

In [7]:
data=pd.read_csv('dataForML.csv',index_col=0)
data.head(2)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,LastNameNum,TicketSibling,isParent,isEnfant,AgeBin,FareBin,IsAlone,FamilySize,Servant,parentSurvived,childrenSurvived,parentDied,childrenDied,SiblingDied,SiblingSurvived
train,1,0.0,3,1,-0.578506,1,0,719,-0.999433,2,4,100,1,0.0,0.0,1,0,0,2,0,0,0,0,0,0,0
train,2,1.0,1,0,0.623779,1,0,815,1.393960,0,5,182,2,0.0,0.0,2,4,0,2,0,0,0,0,0,0,0


In [8]:
data=pd.concat([data, pd.get_dummies(data["Pclass"], prefix='_Pclass_')], axis=1)
data=pd.concat([data, pd.get_dummies(data["Title"], prefix='_Title_')], axis=1)
data=pd.concat([data, pd.get_dummies(data["Embarked"], prefix='_Embarked_')], axis=1)
data=pd.concat([data, pd.get_dummies(data["Ticket"], prefix='_Ticket_')], axis=1)
data=pd.concat([data, pd.get_dummies(data["LastNameNum"], prefix='_LastNameNum_')], axis=1)
data = data.drop(['AgeBin','FareBin','Title','Embarked','LastNameNum','Ticket'],axis=1)

y = data.loc['train','Survived'].astype('int')
X = data.loc['train'].drop(['Survived','PassengerId'],axis=1)
X_cible =  data.loc['test'].drop(['Survived','PassengerId'],axis=1)

In [12]:
from sklearn.ensemble import ExtraTreesClassifier

tree=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
       max_depth=22, max_features='auto', max_leaf_nodes=None,
       min_impurity_decrease=0.0003,
       min_samples_leaf=1, min_samples_split=2,
       min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
       oob_score=False, random_state=42, verbose=0, warm_start=False)

In [13]:
# après quelques tests je me limite à supprimer 'isEnfant'
X=X.drop(['isEnfant'], axis=1)
X_cible=X_cible.drop(['isEnfant'], axis=1)

In [15]:
# Modules KERAS
import keras
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from numpy.random import seed
import tensorflow as tf

# définition de n_cols : la dimension des données d'entrée
n_cols = X.shape[1]

# définition du seed (graine des fonctions d'aléas) afin de pouvoir reproduire à l'identique les résultats des calculs
seed(42)
# Définition de la graine aléatoire pour TensorFlow
tf.random.set_seed(1234)

# Architecture du modèle, un ecouche d'entrée, une de sortie et deux couches "profondes"
# cette architecture est un peu empirique, il n'y a pas vraiment de rêgles permettant de définir
# la meilleur architecture
nb_neurones=47
model = Sequential()
model.add(Dense(nb_neurones, activation='linear', input_shape = (n_cols,)))
model.add(Dense(nb_neurones, activation='linear')) 
model.add(Dense(nb_neurones, activation='linear')) 

# option dropout, cette option évite le surajustement (overfitting) en injectant des aléas sur les données
# d'entrée (une partie des données est aléatoirement fixée à zero)
model.add(Dropout(0.1))
    
# couche de sortie
model.add(Dense(1, activation='sigmoid'))  # output layer

# compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

# exécution du modèle
history = model.fit(X,y,epochs=200,batch_size=512, validation_split=0.2, verbose=0)

TypeError: fit() got an unexpected keyword argument 'nb_epoch'